# Analyser des reçus avec Form Recognizer

![Robot tenant un reçu](./images/receipt_analysis.jpg)

Dans le domaine de l’intelligence artificielle (IA) de la vision par ordinateur, la reconnaissance optique de caractères (OCR) est couramment utilisée pour lire des documents imprimés ou manuscrits. Souvent, le texte est simplement extrait des documents dans un format qui peut être utilisé pour un traitement ou une analyse ultérieure.

Un scénario d’OCR plus avancé est l’extraction d’informations de formulaires, tels que des bons de commande ou des factures, avec une compréhension sémantique de ce que les champs du formulaire représentent. Le service **Form Recognizer** est spécialement conçu pour ce type de problème d’IA.

## Afficher un reçu

Dans cet exemple, vous allez utiliser le modèle intégré de Form Recognizer pour analyser des reçus.

Cliquez sur le bouton **Exécuter la cellule** (&#9655;) (à gauche de la cellule) ci-dessous pour l’exécuter et afficher un exemple de reçu pour lequel vous utiliserez Form Recognizer pour l’analyser.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline


# Load and display a receipt image
fig = plt.figure(figsize=(6, 6))
image_path = os.path.join('data', 'form-receipt', 'receipt.jpg')
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

## Créer une ressource Form Recognizer

Commencez par Créer une ressource Form Recognizer dans votre abonnement Azure :

1. Sous un autre onglet du navigateur, ouvrez le portail Azure à l’adresse https://portal.azure.com, en vous connectant avec votre compte Microsoft.
2. Sélectionnez **+ Créer une ressource**, puis recherchez *Form Recognizer*.
3. Dans la liste des services, sélectionnez **Form Recognizer**.
4. Dans le panneau **Form Recognizer**, sélectionnez **Créer**.
5. Dans le panneau **Créer**, entrez les détails suivants et sélectionnez **Créer**
   - **Nom** : Nom unique de votre service
   - **Abonnement** : Votre abonnement Azure
   - **Région** : Région disponible
   - **Niveau tarifaire** : F0
   - **Groupe de ressources** : Groupe de ressources existant que vous avez utilisé précédemment
   - **Je confirme avoir lu et compris l’avis ci-dessous** : Sélectionné.
6. Attendez que le service soit créé.
7. Affichez votre service Form Recognizer nouvellement créé dans le portail Azure et sur la page **Clés et point de terminaison**. Copiez ensuite les valeurs **Clé1** et **Point de terminaison** et collez-les dans la cellule de code ci-dessous, en remplaçant les valeurs **YOUR_FORM_KEY** et **YOUR_FORM_ENDPOINT**.

In [ ]:
form_key = 'YOUR_FORM_KEY'
form_endpoint = 'YOUR_FORM_ENDPOINT'

print('Ready to use form recognizer at {} using key {}'.format(form_endpoint, form_key))

## Analyser un reçu

Vous êtes maintenant prêt à utiliser Form Recognizer pour analyser un reçu.

In [ ]:
import os
from azure.ai.formrecognizer import FormRecognizerClient
from azure.core.credentials import AzureKeyCredential

# Create a client for the form recognizer service
form_recognizer_client = FormRecognizerClient(endpoint=form_endpoint, credential=AzureKeyCredential(form_key))

try:
    print("Analyzing receipt...")
    # Get the receipt image file
    image_path = os.path.join('data', 'form-receipt', 'receipt.jpg')

    # Submit the file data to form recognizer
    with open(image_path, "rb") as f:
        analyze_receipt = form_recognizer_client.begin_recognize_receipts(receipt=f)
    
    # Get the results
    receipt_data = analyze_receipt.result()

    # Print the extracted data for the first (and only) receipt
    receipt = receipt_data[0]
    receipt_type = receipt.fields.get("ReceiptType")
    if receipt_type:
        print("Receipt Type: {}".format(receipt_type.value))
    merchant_address = receipt.fields.get("MerchantAddress")
    if merchant_address:
        print("Merchant Address: {}".format(merchant_address.value))
    merchant_phone = receipt.fields.get("MerchantPhoneNumber")
    if merchant_phone:
        print("Merchant Phone: {}".format(merchant_phone.value))
    transaction_date = receipt.fields.get("TransactionDate")
    if transaction_date:
        print("Transaction Date: {}".format(transaction_date.value))
    print("Receipt items:")
    items = receipt.fields.get("Items")
    if items:
        for idx, item in enumerate(receipt.fields.get("Items").value):
            print("\tItem #{}".format(idx+1))
            item_name = item.value.get("Name")
            if item_name:
                print("\t - Name: {}".format(item_name.value))
            item_total_price = item.value.get("TotalPrice")
            if item_total_price:
                print("\t - Price: {}".format(item_total_price.value))
    subtotal = receipt.fields.get("Subtotal")
    if subtotal:
        print("Subtotal: {} ".format(subtotal.value))
    tax = receipt.fields.get("Tax")
    if tax:
        print("Tax: {}".format(tax.value))
    total = receipt.fields.get("Total")
    if total:
        print("Total: {}".format(total.value))

except Exception as ex:
    print('Error:', ex)


Notez que Form Recognizer est capable d’interpréter les données du formulaire, en identifiant correctement l’adresse et le numéro de téléphone du commerçant, la date et l’heure de la transaction, ainsi que les articles de ligne, le sous-total, la taxe applicable et les montants totaux.

## Informations complémentaires

Pour plus d’informations sur le service Form Recognizer, consultez [la documentation de Form Recognizer](https://docs.microsoft.com/fr-fr/azure/cognitive-services/form-recognizer/index)